In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset
from tqdm import tqdm
import random
import os

In [3]:
device = 'cuda'

In [4]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
seed_everything(42)

# Dataset

In [6]:
df_train = pd.read_csv('./data/train_result_mean_median_max_min_current_next_prev.csv')
df_test = pd.read_csv('./data/test_result_mean_median_max_min_current_next_prev.csv')

In [7]:
df_train.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_next_min,X4_next_min,X2_prev_min,X4_prev_min,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,780,905,867,931,954,935,40,1,2,7
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2110,2096,2224,2126,2026,2034,17,13,8,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1822,2153,2092,2047,1883,2053,1,1,2,4
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1416,1506,1564,1502,1474,1,1,1,26
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1263,1264,1239,1162,1202,1191,8,10,19,1


In [8]:
fig_train = df_train.iloc[:, 6:22].as_matrix()
fig_test = df_test.iloc[:, 6:22].as_matrix()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [9]:
df_train['1/to_next_game_1'] = df_train['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_next_game_2'] = df_train['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_1'] = df_train['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_2'] = df_train['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

df_test['1/to_next_game_1'] = df_test['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_next_game_2'] = df_test['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_1'] = df_test['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_2'] = df_test['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

In [10]:
df_train

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2,1/to_next_game_1,1/to_next_game_2,1/to_prev_game_1,1/to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,954,935,40,1,2,7,0.025000,1.000000,0.500000,0.142857
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2026,2034,17,13,8,1,0.058824,0.076923,0.125000,1.000000
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1883,2053,1,1,2,4,1.000000,1.000000,0.500000,0.250000
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1474,1,1,1,26,1.000000,1.000000,1.000000,0.038462
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1202,1191,8,10,19,1,0.125000,0.100000,0.052632,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1211,1049,6,0,16,166,0.166667,0.000000,0.062500,0.006024
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,699,706,0,18,4,10,0.000000,0.055556,0.250000,0.100000
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1063,1036,5,1,257,0,0.200000,1.000000,0.003891,0.000000
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1752,1719,0,1,4,1,0.000000,1.000000,0.250000,1.000000


In [11]:
rating_1_train = df_train[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_train = df_train[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

rating_1_test =  df_test[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_test = df_test[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

In [12]:
y_train = df_train['target']
rating_1_train, rating_2_train, fig_train, y_train = shuffle(rating_1_train, rating_2_train, fig_train, y_train, random_state=42)

In [13]:
rating_1_train, rating_1_val, \
rating_2_train, rating_2_val, \
fig_train, fig_val, \
y_train, y_val = train_test_split(rating_1_train, rating_2_train, fig_train, y_train, random_state=42, test_size=0.2)

In [14]:
rating_train = np.concatenate([rating_1_train, rating_2_train], axis=1)
rating_val = np.concatenate([rating_1_val, rating_2_val], axis=1)
rating_test = np.concatenate([rating_1_test, rating_2_test], axis=1)

# Data for torch

In [15]:
class TrainDataset(Dataset):
    def __init__(self, figs, rating, y):
        self.figs = torch.tensor(figs, dtype=torch.long)
        self.rating = torch.tensor(rating, dtype=torch.float)
        self.y = torch.tensor(y.tolist(), dtype=torch.float)
    
    def __len__(self):
        return len(self.figs)
    
    def __getitem__(self, idx):
        return {
            'figs': self.figs[idx],
            'rating': self.rating[idx],
            'y': self.y[idx]
        }

class TestDataset(Dataset):
    def __init__(self, figs, rating):
        self.figs = torch.tensor(figs, dtype=torch.long)
        self.rating = torch.tensor(rating, dtype=torch.float)
    
    def __len__(self):
        return len(self.figs)
    
    def __getitem__(self, idx):
        return {
            'figs': self.figs[idx],
            'rating': self.rating[idx]
        }
        

In [16]:
def train_epoch(iterator, optimizer, criterion):
        model.train()
        epoch_loss = 0

        for i, batch in tqdm(enumerate(iterator)):
            targets = batch['y'].to(device)
            figs = batch['figs'].to(device)
            rating = batch['rating'].to(device)

            optimizer.zero_grad()

            logits = model(figs, rating).squeeze()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def val_epoch(iterator,criterion):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                targets = batch['y'].to(device)
                figs = batch['figs'].to(device)
                rating = batch['rating'].to(device)

                logits = model(figs, rating).squeeze()
                loss = criterion(logits, targets)
                epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def predict(iterator):
        model.eval()
        preds = []
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                figs = batch['figs'].to(device)
                rating = batch['rating'].to(device)
                pred = model.predict(figs, rating)
                preds += pred.to('cpu').numpy().tolist()

        return preds

In [17]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.embdedding = nn.Embedding(101, 32)
        self.fc1figs = nn.Linear(32*8*2, 128)
        self.bn1figs = nn.BatchNorm1d(128)        
        self.fc2figs = nn.Linear(128, 128)
        self.bn2figs = nn.BatchNorm1d(128)
        self.fc3figs = nn.Linear(128, 64)
        self.bn3figs = nn.BatchNorm1d(64)
       
        self.fc1rating = nn.Linear(34, 64)
        self.bn1rating = nn.BatchNorm1d(64)
        self.fc2rating = nn.Linear(64, 32)
        self.bn2rating = nn.BatchNorm1d(32)
        self.fc3rating = nn.Linear(32, 16)
        self.bn3rating = nn.BatchNorm1d(16)
        
        self.do = nn.Dropout(0.1)
        self.fc1res = nn.Linear(64+16, 16)
        self.bn1res = nn.BatchNorm1d(16)
        self.fc2res = nn.Linear(16, 1)
    
    def predict(self, figs, rating):
        result = self.forward(figs, rating)
        result = F.sigmoid(result)
        return result
        
    def forward(self, figs, rating):
        figs = self.embdedding(figs)
        figs = figs.reshape(-1, 32*8*2)
        figs = self.do(figs)
        figs = self.fc1figs(figs)
        figs = self.bn1figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        figs = self.fc2figs(figs)
        figs = self.bn2figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        figs = self.fc3figs(figs)
        figs = self.bn3figs(figs)
        figs = F.relu(figs)
        #figs = self.do(figs)
        
        
        rating = self.fc1rating(rating)
        rating = self.bn1rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        rating = self.fc2rating(rating)
        rating = self.bn2rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        rating = self.fc3rating(rating)
        rating = self.bn3rating(rating)
        rating = F.relu(rating)
        #rating = self.do(rating)
        
        concat_data = torch.cat((figs, rating), dim=-1)
        
        result = self.fc1res(concat_data)
        result = self.bn1res(result)
        result = F.relu(result)
        result = self.fc2res(result)
        
        return result

In [18]:
batch_size = 64
train_dataset = TrainDataset(fig_train, rating_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TrainDataset(fig_val, rating_val, y_val)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = TestDataset(fig_test, rating_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [19]:
train_dataset[0]

{'figs': tensor([  1,  79,  26,  89, 100,  50,  80,  18,  96,  25,  80,  98,  79,  16,
          71,  37]),
 'rating': tensor([1.0470e+03, 9.8700e+02, 1.0750e+03, 1.0470e+03, 9.8700e+02, 1.0750e+03,
         1.0470e+03, 9.8700e+02, 1.0750e+03, 1.0470e+03, 9.8700e+02, 1.0750e+03,
         1.0470e+03, 9.3000e+01, 1.0000e+00, 1.0753e-02, 1.0000e+00, 1.0550e+03,
         1.0600e+03, 9.4300e+02, 1.0550e+03, 1.0600e+03, 9.4300e+02, 1.0550e+03,
         1.0600e+03, 9.4300e+02, 1.0550e+03, 1.0600e+03, 9.4300e+02, 1.0550e+03,
         3.1200e+02, 3.4200e+02, 3.2051e-03, 2.9240e-03]),
 'y': tensor(0.)}

In [20]:
model = NN()
if device == 'cuda':
    model = model.cuda()

In [21]:
epochs = 50
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7, last_epoch=-1)

In [22]:
best_valid_loss = float('inf')
for epoch in range(1, epochs + 1):
    train_loss = round(train_epoch(train_dataloader, optimizer, criterion), 7)
    valid_loss = round(val_epoch(valid_dataloader, criterion), 7)
    if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), './data/models/result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_add_order.pt')

    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    scheduler.step()

45413it [05:21, 141.40it/s]
11354it [00:15, 709.92it/s]
0it [00:00, ?it/s]

Epoch: 1 
 Train Loss 0.5355818  Val loss 0.5329477:


45413it [05:23, 140.49it/s]
11354it [00:15, 726.21it/s]
0it [00:00, ?it/s]

Epoch: 2 
 Train Loss 0.50988  Val loss 0.6871411:


45413it [05:13, 144.80it/s]
11354it [00:15, 743.63it/s]
1it [00:00,  5.56it/s]

...

Epoch: 48 
 Train Loss 0.4564438  Val loss 0.4640676:


45413it [04:54, 154.07it/s]
11354it [00:14, 765.40it/s]
1it [00:00,  5.53it/s]

Epoch: 49 
 Train Loss 0.4563364  Val loss 0.4649784:


45413it [04:44, 159.42it/s]
11354it [00:14, 767.77it/s]

Epoch: 50 
 Train Loss 0.456455  Val loss 0.4651481:


In [23]:
model.load_state_dict(torch.load('./data/models/result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_add_order.pt'))

<All keys matched successfully>

In [24]:
round(val_epoch(valid_dataloader, criterion), 7)

11354it [00:14, 771.06it/s]


0.4640676

In [25]:
res = predict(test_dataloader)

0it [00:00, ?it/s]/home/mpopov/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
14192it [00:15, 902.39it/s]


In [26]:
res = [item[0] for item in res]

In [27]:
df_test['target'] = res

In [28]:
df_test[['id','target']].to_csv('result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_add_order.csv', index=False)

In [29]:
!head  result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_add_order.csv

id,target
0,0.5109214186668396
1,0.015112150460481644
2,0.2812189757823944
3,0.11097893118858337
4,0.08722084760665894
5,0.05714172497391701
6,0.5541698932647705
7,0.2499494105577469
8,0.8924974203109741


# val_data

In [30]:
val_preds = predict(valid_dataloader)
val_preds = [item[0] for item in val_preds]
df_val = pd.DataFrame({'score':val_preds, 'target': y_val})

11354it [00:13, 828.87it/s]


In [31]:
df_val.to_csv('val_result_more_weights_more_layers_figures_without_order_NN_with_means_median_max_min_dist_no_dropout_add_order.csv', index=False)